### Google Calendar API 연동
- Google Calendar에 대한 학습보다는 Function Call 기능을 위한 Function 이해도 증가 목적

- Google Calendar API 셋업 필요
  - https://developers.google.com/calendar/api/quickstart/python?hl=ko
  - 위 가이드를 따라가도 되지만 아래 직접 작성한 API 셋업 방법 참고하시는 것을 권장

### Google Calendar API 셋업 방법
1. Google Cloud Project 생성
    - https://console.cloud.google.com/projectcreate
2. Google Calendar API 사용 설정 및 API Key 발급
    - 라이브러리 > Calendar 검색 > Google Calendar "사용" 설정
        - https://console.cloud.google.com/flows/enableapi?apiid=calendar-json.googleapis.com&hl=ko
    - 메뉴 > API 및 서비스 > 사용자 인증 정보 > 사용자 인증 정보 만들기 > API 키 (*공식 가이드엔 이 내용이 없음)
3. OAuth 동의 화면 생성 및 테스트 유저에 본인 추가
    - 메뉴 > OAuth 동의 화면
        - User Type: 외부 > 만들기
        - 필수 항목 작성
    - 범위 추가
        - .../auth/calendar.events.owned > 업데이트
    - 테스트 사용자로 본인 추가
4. OAuth Client ID 생성
    - 메뉴 > 사용자 인증 정보 > 사용자 인증 정보 만들기 > OAuth 클라이언트 ID
    - 애플리케이션 유형: 데스크톱 앱
    - "OAuth 클라이언트 생성됨" 화면이 뜨면 JSON 다운로드 버튼 클릭
    - 다운로드 받은 파일을 ./res/credentials.json으로 저장

### 잘 진행되었다면 아래 코드가 정상적으로 수행되어야 함
- 구글 계정 로그인 후 "Google에서 확인하지 않은 앱 - 현재 테스트 중인 앱에 대한 액세스 권한을 받았습니다" 내용
  - 계속 버튼 클릭
- 이 확인 프로세스는 첫 1회만 수행 (아래 코드가 ./res/token.json 저장 후 자동 인증)

In [2]:
import os.path

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build


SCOPES = ['https://www.googleapis.com/auth/calendar']


# 만약 invalid_grant: Token has been expired or revoked 에러 발생 시 ./res/token.json을 지우고 다시 코드 실행해서 ./res/token.json 재생성
if os.path.exists('./res/token.json'):
    creds = Credentials.from_authorized_user_file('./res/token.json', SCOPES)
else:
    flow = InstalledAppFlow.from_client_secrets_file(
        './res/credentials.json', SCOPES
    )
    creds = flow.run_local_server(port=0)
    with open('./res/token.json', 'w') as token:
        token.write(creds.to_json())

service = build('calendar', 'v3', credentials=creds)

### 캘린더 일정 생성

In [3]:
event = {
  'summary': '패스트캠퍼스 LLM 강의',
  'location': '서울 강남구 테헤란로 231 West동 6층, 7층',
  'start': {
    'dateTime': '2024-08-14T09:00:00+09:00',
    'timeZone': 'Asia/Seoul',
  },
  'end': {
    'dateTime': '2024-08-14T18:00:00+09:00',
    'timeZone': 'Asia/Seoul',
  }
}

event = service.events().insert(calendarId='primary', body=event).execute()
print('Event created: %s' % (event.get('htmlLink')))

Event created: https://www.google.com/calendar/event?eid=MGVsanY0bDI3MzNmZzUxbDF1M3RiMm5mdnMgcmVpbjIwLmFpQG0


In [4]:
event

{'kind': 'calendar#event',
 'etag': '"3447270646198000"',
 'id': '0eljv4l2733fg51l1u3tb2nfvs',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=MGVsanY0bDI3MzNmZzUxbDF1M3RiMm5mdnMgcmVpbjIwLmFpQG0',
 'created': '2024-08-14T11:35:23.000Z',
 'updated': '2024-08-14T11:35:23.099Z',
 'summary': '패스트캠퍼스 LLM 강의',
 'location': '서울 강남구 테헤란로 231 West동 6층, 7층',
 'creator': {'email': 'rein20.ai@gmail.com', 'self': True},
 'organizer': {'email': 'rein20.ai@gmail.com', 'self': True},
 'start': {'dateTime': '2024-08-14T09:00:00+09:00', 'timeZone': 'Asia/Seoul'},
 'end': {'dateTime': '2024-08-14T18:00:00+09:00', 'timeZone': 'Asia/Seoul'},
 'iCalUID': '0eljv4l2733fg51l1u3tb2nfvs@google.com',
 'sequence': 0,
 'reminders': {'useDefault': True},
 'eventType': 'default'}

### 캘린더 일정 조회

In [5]:
calendar_id = 'primary'
time_min = '2024-08-14T00:00:00+09:00'
time_max = '2024-08-14T23:59:59+09:00'
max_results = 5
is_single_events = True
orderby = 'startTime'

events_result = service.events().list(calendarId = calendar_id,
                                      timeMin = time_min,
                                      timeMax = time_max,
                                      maxResults = max_results,
                                      singleEvents = is_single_events,
                                      orderBy = orderby
                                     ).execute()

In [6]:
events_result

{'kind': 'calendar#events',
 'etag': '"p33sat7vpm7q8e0o"',
 'summary': 'rein20.ai@gmail.com',
 'description': '',
 'updated': '2024-08-14T11:35:23.099Z',
 'timeZone': 'Asia/Seoul',
 'accessRole': 'owner',
 'defaultReminders': [{'method': 'popup', 'minutes': 30}],
 'items': [{'kind': 'calendar#event',
   'etag': '"3447270646198000"',
   'id': '0eljv4l2733fg51l1u3tb2nfvs',
   'status': 'confirmed',
   'htmlLink': 'https://www.google.com/calendar/event?eid=MGVsanY0bDI3MzNmZzUxbDF1M3RiMm5mdnMgcmVpbjIwLmFpQG0',
   'created': '2024-08-14T11:35:23.000Z',
   'updated': '2024-08-14T11:35:23.099Z',
   'summary': '패스트캠퍼스 LLM 강의',
   'location': '서울 강남구 테헤란로 231 West동 6층, 7층',
   'creator': {'email': 'rein20.ai@gmail.com', 'self': True},
   'organizer': {'email': 'rein20.ai@gmail.com', 'self': True},
   'start': {'dateTime': '2024-08-14T09:00:00+09:00',
    'timeZone': 'Asia/Seoul'},
   'end': {'dateTime': '2024-08-14T18:00:00+09:00', 'timeZone': 'Asia/Seoul'},
   'iCalUID': '0eljv4l2733fg51l1u3tb

### 캘린더 일정 삭제

In [7]:
event = service.events().delete(calendarId='primary', eventId=event['id']).execute()
print('Event deleted')

Event deleted


In [8]:
event

''